# Measurement of thermalization

There are at least two ways to measure thermalization, the spectral entropy, and participation number.

## Spectral Entropy

Spectral entropy is given by
$$S = -\sum_i e_i \ln(e_i),$$

with $e_i/E$ the fractional energy in the $i$th mode. Using constrained optimization with the constraint

$$\sum_i e_i = 1,$$ 

one can show that this is maximized when all modes have equal energy $e_i = 1/N$, assuming all modes can be accessed by the non-linearity from a given initial condition. As Pace and Campbell noted in their 2019 paper, this has to be modified for the $\beta$ FPUT model because in that case, energy cannot be exchanged between even and odd modes. Since the IC for the standard FPUT model has $e_1 = E$, $S_{max} = \lceil N/2 \rceil$. 

For us, what does this mean? Our non-linear term is mostly the same as the $\alpha$ FPUT, but it does include the alternating $k_i$. The eigenmodes of the linear operator are no longer $\propto \sin(n \pi q_i)$ where $q_i$ is the displacement from equilibrium of the $i$th mass. While the $\sin(n\pi q)$ modes are still an orthonormal basis, so if they are all accessible from the initial condition, then the maximum entropy remains the same.  **However**, even a single eigenmode of our model is a mixture of all $\sin$ modes (or perhaps only the even/odd ones. 

The question is: does a single sine mode stay a single side mode? Of course it shouldn't, it's not a stationary state.

In [1]:
%matplotlib widget
import h5py
import numpy as np
import matplotlib.pyplot as plt
import utils
import scipy.fftpack as fpack
from matplotlib.widgets import Slider, Button
plt.style.use('prl')

First, load data from run B, which is a linear ($\alpha = 0$), topologically protected run.

In [2]:
class fput_run:
    def __init__(self, filename):
        self.fn = filename
        df = h5py.File(self.fn, "r")
        self.p = df['tasks/p'][:]
        self.q = df['tasks/q'][:]
        self.t = df['scales/t'][:]

        self.et = df['energies/e_tot'][:]
        self.e1 = df['energies/e_1'][:]
        self.e2 = df['energies/e_2'][:]
        self.e1m = df['energies/e_1_emode'][:]
        self.e2m = df['energies/e_2_emode'][:]
        df.close()

In [3]:
run_B = fput_run("data/run_B_output.h5")
run_C = fput_run("data/run_C_output.h5")

Next, construct its linear operator and calculate the eigenmodes and frequencies. 

In [4]:
ko = 0.5
ke = 1
N = 32
H = utils.linear_operator(N, k1=ko, k2=ke)
omega, evecs = utils.efreq(H)

Project each $p_n(t)$, $q_n(t)$ against them, making sure to adjust $\hat{q}_n(t)$ for its temporal part, $\cos(\omega_n t)$. Note that because the eigenvectors are for $q$, to get $p = \dot{q}$ (since $m=1$), one multiplies by $-\omega_n \sin(\omega_n t)$. So, we adjust $p$ appropriately. Note that the $\omega_n$ factor is recovered exactly if one solves the first-order formulation of the equations of motion. In that case, the eigenvectors return both $q_n$ *and* $p_n$, verifying the relationship.

In [11]:
def emode_vs_t(p, q, t, evecs, omega):
    p_hat = np.zeros_like(p[:,1:-1])
    q_hat = np.zeros_like(q[:,1:-1])
    for i in range(p.shape[0]):
        p_hat[i,:], q_hat[i,:] = utils.eigenmode_transform(p[i,1:-1],q[i,1:-1],evecs)
        p_hat[i,:] *= omega/np.sin(omega*t[i])
        q_hat[i,:] /= np.cos(omega*t[i])
        
    return p_hat, q_hat

In [12]:
plt.figure()
plt.subplot(211)
plt.imshow(run_B.q.T,extent=(0,100,0,31),origin='lower',interpolation='none')
plt.colorbar()
plt.subplot(212)
plt.imshow(np.abs(run_B.p.T),extent=(0,100,0,31),origin='lower',interpolation='none')
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Now, compute the emode transform as well as the DST, to get the sine mode amplitudes for $q_n$. Note that because we use "ghost masses" outside the $N$ degree-of-freedom grid (that is, mass 0 and mass $N$ are fixed in position), if we restrict only to the $N$ degrees of freedom, our signal is odd about $-1$ and $N$, so we need the DST-Type I.

In [13]:
run_B.phat, run_B.qhat = emode_vs_t(run_B.p,run_B.q,run_B.t, evecs, omega)
run_B.q_dst = fpack.dst(run_B.q[:,1:-1],type=1)

run_C.phat, run_C.qhat = emode_vs_t(run_C.p,run_C.q,run_C.t, evecs, omega)
run_C.q_dst = fpack.dst(run_C.q[:,1:-1],type=1)

<ipython-input-11-17a4cca76838>:6: RuntimeWarning: divide by zero encountered in true_divide
  p_hat[i,:] *= omega/np.sin(omega*t[i])
<ipython-input-11-17a4cca76838>:6: RuntimeWarning: invalid value encountered in multiply
  p_hat[i,:] *= omega/np.sin(omega*t[i])


We can see that the eigenmodes are constant, at $n=15$, which is what we initialized run B with:

In [14]:
plt.figure()
plt.subplot(211)
plt.imshow(run_B.qhat.T,extent=(0,100,0,31),origin='lower',interpolation='none')
plt.colorbar()
plt.subplot(212)
plt.imshow(np.abs(run_B.phat.T),extent=(0,100,0,31),origin='lower',interpolation='none')
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

However, all odd-indexed $\sin$ modes are excited. Have to be careful here, since mode number 0 is probably $\sin(\pi n/(N+1))$, so the odd *indexed* modes are probably the *even* modes...

In [21]:
plt.figure()
plt.imshow(run_B.q_dst.T,extent=(0,100,0,31),interpolation='none')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
plt.figure()
plt.plot(run_B.t, run_B.q_dst[:,13],label='mode 13')
plt.plot(run_B.t, run_B.q_dst[:,15],label='mode 15')
plt.plot(run_B.t,run_B.q_dst[:,16], label='mode 16')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Run C is a linear, protected run with $n=15$ $\sin$ mode as the initial conditon. Here, we look at the time evolution of the $\sin$ amplitudes, and as expected, many of them participate in complex ways.

In [23]:
plt.figure()
plt.imshow(run_C.q_dst.T,extent=(0,100,0,31),interpolation='none')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
plt.figure()
plt.plot(run_C.t, run_C.q_dst[:,12],label='mode 12')
plt.plot(run_C.t, run_C.q_dst[:,14],label='mode 14')
plt.plot(run_C.t,run_C.q_dst[:,16], label='mode 16')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Maybe we could calculate the entropy using the *eigenmodes* themselves?** As we can see below, the combination of eigenmodes excited by the $\sin$ mode is fixed in time, though it is not, of course, at minimum entropy.

In [25]:
plt.figure()
plt.subplot(211)
plt.imshow(run_C.qhat.T,extent=(0,100,0,31),origin='lower',interpolation='none')
plt.colorbar()
plt.subplot(212)
plt.imshow(np.abs(run_C.phat.T),extent=(0,100,0,31),origin='lower',interpolation='none')
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Gravest eigenmodes

How do $n=1$ modes compare between $\mathrm{To} = 0$, $\mathrm{To} < 0$, $\mathrm{To} > 0$?

In [15]:
n = np.arange(1,N+1)
# Define initial parameters
init_mode = 1

# Create the figure and the line that we will manipulate
fig, ax = plt.subplots()
line1, = plt.plot(np.sin(np.pi*init_mode*n/(N+1)), 'o-', alpha=0.5, lw=2, label='sin mode {:d}'.format(init_mode))
line2, = plt.plot(evecs[:,init_mode-1]/np.abs(evecs[:,init_mode-1]).max(), 'o-', alpha=0.5, lw=2, label='eigenmode {:d}'.format(init_mode))
ax.legend()
ax.set_ylim(-1,1)
ax.set_xlabel('Mass')

# adjust the main plot to make room for the sliders
plt.subplots_adjust(bottom=0.25)

# Make a horizontal slider to control the frequency.
axfreq = plt.axes([0.25, 0.1, 0.65, 0.03])
freq_slider = Slider(
    ax=axfreq,
    label='mode number',
    valmin=1,
    valmax=31,
    valfmt='%0.0f',
    valinit=init_mode,
)


# The function to be called anytime a slider's value changes
def update(val):
    mode= np.int(val)
    line1.set_label('sin mode {:d}'.format(mode))
    line2.set_label('eigenmode {:d}'.format(mode))
    ax.legend()
    line1.set_ydata(np.sin(np.pi*mode*n/(N+1)))
    line2.set_ydata(evecs[:,mode-1]/np.abs(evecs[:,mode-1]).max())
    fig.canvas.draw_idle()


# register the update function with each slider
freq_slider.on_changed(update)

# Create a `matplotlib.widgets.Button` to reset the sliders to initial values.
resetax = plt.axes([0.8, 0.025, 0.1, 0.04])
button = Button(resetax, 'Reset', hovercolor='0.975')


def reset(event):
    freq_slider.reset()
    amp_slider.reset()
button.on_clicked(reset)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
print("Run B Energy = {}".format(run_B.et[0]))
print("Run C Energy = {}".format(run_C.et[0]))

Run B Energy = 0.7499942779540857
Run C Energy = 11.474398794500468


## Participation number

The 

# Old Stuff
Old stuff to look at entropy contribution from a single mode...

In [16]:
e = np.logspace(0,-10,1000)

In [17]:
s = -e*np.log(e)

In [18]:
plt.semilogx(e,s)
plt.axhline(-np.log(1/np.exp(1))/np.exp(1), alpha=0.4)
plt.xlabel("Fractional energy")
plt.ylabel("Entropy Contribution of a mode")

Text(0, 0.5, 'Entropy Contribution of a mode')